In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
import onnx
import onnxruntime

In [2]:
def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )

In [3]:
class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers = []
        if expand_ratio != 1:
            layers.append(nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False))
            layers.append(nn.BatchNorm2d(hidden_dim))
            layers.append(nn.ReLU6(inplace=True))
        layers.extend([
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup),
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

In [4]:
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=10, width_mult=1.):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        inverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        input_channel = int(input_channel * width_mult)
        self.last_channel = int(last_channel * max(1.0, width_mult))
        features = [conv_bn(1, input_channel, 2)]  # Changed input channels to 1 for MNIST
        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = int(c * width_mult)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        features.append(conv_bn(input_channel, self.last_channel, 1))
        # make it nn.Sequential
        self.features = nn.Sequential(*features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
        x = x.mean([2, 3])
        x = self.classifier(x)
        return x

In [5]:
def preprocess_data(device, batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    
    return train_loader, test_loader

In [6]:
def calculate_accuracy(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in data_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return 100 * correct / total

In [7]:
def train(model, train_loader, criterion, optimizer, device, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            if batch_idx % 100 == 99:  # print every 100 mini-batches
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
                running_loss = 0.0
        
        # Calculate and print training accuracy at the end of each epoch
        train_accuracy = calculate_accuracy(model, train_loader, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.2f}%')

In [8]:
def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')
    return accuracy

In [9]:
def save_model(model, path='mobilenet_mnist.pth'):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [10]:
def load_model(model, path='mobilenet_mnist.pth', device='cpu'):
    if os.path.exists(path):
        model.load_state_dict(torch.load(path))
        print(f"Model loaded from {path}")
        return True
    return False

In [11]:
def export_to_onnx(model, sample_input, onnx_path='mobilenet_mnist.onnx'):
    torch.onnx.export(model, sample_input, onnx_path, export_params=True, opset_version=10, do_constant_folding=True, input_names=['input'], output_names=['output'], dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
    print(f"Model exported to ONNX format at {onnx_path}")

In [12]:
def verify_onnx(onnx_path='mobilenet_mnist.onnx'):
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    print("ONNX model is valid")

In [13]:
def test_onnx(onnx_path, test_loader, device):
    session = onnxruntime.InferenceSession(onnx_path, providers=['CPUExecutionProvider'] if device == 'cpu' else ['CUDAExecutionProvider'])
    
    correct = 0
    total = 0
    for data, targets in test_loader:
        data = data.numpy()  # Convert to numpy array
        outputs = session.run(None, {'input': data})
        predicted = outputs[0].argmax(axis=1)
        total += targets.size(0)
        correct += (predicted == targets.numpy()).sum()
    
    accuracy = 100 * correct / total
    print(f'ONNX model accuracy on test set: {accuracy:.2f}%')

In [14]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    train_loader, test_loader = preprocess_data(device)
    
    model = MobileNetV2().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    num_epochs = 10
    model_path = 'mobilenetv2_mnist.pth'
    onnx_path = 'mobilenetv2_mnist.onnx'
    
    if not load_model(model, model_path, device):
        print("Training new model...")
        train(model, train_loader, criterion, optimizer, device, num_epochs)
        save_model(model, model_path)
    else:
        print("Using pre-trained model.")
    
    test(model, test_loader, device)
    
    # Export to ONNX
    sample_input = torch.randn(1, 1, 28, 28).to(device)
    export_to_onnx(model, sample_input, onnx_path)
    verify_onnx(onnx_path)
    test_onnx(onnx_path, test_loader, device)

In [15]:
if __name__ == "__main__":
    main()

Using device: cuda
Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Training new model...
Epoch [1/10], Step [100/938], Loss: 1.5688
Epoch [1/10], Step [200/938], Loss: 0.7146
Epoch [1/10], Step [300/938], Loss: 0.4681
Epoch [1/10], Step [400/938], Loss: 0.3754
Epoch [1/10], Step [500/938], Loss: 0.2884
Epoch [1/10], Step [600/938], Loss: 0.2495
Epoch [1/10], Step [700/938], Loss: 0.2287
Epoch [1/10], Step [800/938], Loss: 0.2115
Epoch [1/10], Step [900/938], Loss: 0.1865
Epoch [1/10], Train Accuracy: 96.07%
Epoch [2/10], Step [100/938], Loss: 0.1748
Epoch [2/10], Step [200/938], Loss: 0.1594
Epoch [2/10], Step [300/938], Loss: 0.1525
Epoch [2/10], Step [400/938], Loss: 0.1353
Epoch [2/10], Step [500/938], Loss: 0.1457
Epoch [2/10], Step [600/938], Loss: 0.1323
Epoch [2/10], Step [700/938], Loss: 0.1369
Epoch [2/10], Step [800/938], Loss: 0.1265
Epoch [2/10], Step [900/938], Loss: 0.1172
Epoch [2/10], Train Accuracy: 97.65%
Epoch [3/10], Step [100/938], Loss: 0.1014
Epoch [3/10]

AttributeError: module 'onnxruntime' has no attribute 'InferenceSession'